# Dirichlet prior as database

BNLearner gives access of many priors for the parameters and structural learning. One of them is the Dirichlet prior which needs a a prior for every possible parameter in a BN. aGrUM/pyAgrum allows to use a database as a source of Dirichlet prior.

In [1]:
%matplotlib inline
from pylab import *
import matplotlib.pyplot as plt

import os

import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

sizePrior=10000
sizeData=30000

## generating databases for Dirichlet prior and for the learning 

In [8]:
bnPrior = gum.fastBN("A->B;C;D")
gum.generateCSV(bnPrior, "dirichlet.csv", sizePrior, with_labels=True,random_order=True)

bnData = gum.fastBN("A->B->C->D")
gum.generateCSV(bnData, "database.csv", sizeData, with_labels=True,random_order=False)

gnb.sideBySide(bnData,bnPrior,
               captions=[f"Database ({sizeData} cases)",f"Prior ({sizePrior} cases)"])

G <!-- A --> A A <!-- B --> B B <!-- A->B --> A->B <!-- C --> C C <!-- B->C --> B->C <!-- D --> D D <!-- C->D --> C->D,G <!-- A --> A A <!-- B --> B B <!-- A->B --> A->B <!-- C --> C C <!-- D --> D D
Database (30000 cases),Prior (10000 cases)


## Learning databases

In [9]:
# bnPrior is used to give the variables and their domains
learnerData = gum.BNLearner("database.csv") 
learnerPrior = gum.BNLearner("dirichlet.csv") 
learnerData.useScoreBIC()
learnerPrior.useScoreBIC()
gnb.sideBySide(learnerData.learnBN(),learnerPrior.learnBN(),
              captions=["Learning from Data","Learning from Prior"])

G <!-- A --> A A <!-- B --> B B <!-- B->A --> B->A <!-- C --> C C <!-- C->B --> C->B <!-- D --> D D <!-- D->C --> D->C,G <!-- C --> C C <!-- D --> D D <!-- B --> B B <!-- A --> A A <!-- A->B --> A->B
Learning from Data,Learning from Prior


## Learning with Dirichlet prior

In [10]:
def learnWithRatio(ratio):
    # bnPrior is used to give the variables and their domains
    learner = gum.BNLearner("database.csv") #, bnPrior) 
    learner.useAprioriDirichlet("dirichlet.csv")
    learner.setAprioriWeight(ratio*sizePrior)
    learner.setDatabaseWeight((1-ratio)) #*sizeData)
    learner.useScoreBIC() # or another score with no included prior
    return learner.learnBN()

ratios=[0.0,1.0] #0.01,0.05,0.2,0.5,0.8,0.9,0.95,0.99,1.0]
gnb.sideBySide(*[learnWithRatio(r) for r in ratios],
              captions=[*[f"with ratio {r}<br/> [datasize : {r*sizePrior+(1-r)*sizeData}]" for r in ratios]])

G <!-- A --> A A <!-- B --> B B <!-- B->A --> B->A <!-- C --> C C <!-- C->B --> C->B <!-- D --> D D <!-- D->C --> D->C,G <!-- A --> A A <!-- B --> B B <!-- C --> C C <!-- D --> D D <!-- C->D --> C->D
with ratio 0.0 [datasize : 30000.0],with ratio 1.0 [datasize : 10000.0]


In [23]:
def learnWithRatio(ratio):
    # bnPrior is used to give the variables and their domains
    learner = gum.BNLearner("database.csv", bnPrior) 
    learner.useAprioriDirichlet("dirichlet.csv")
    learner.setAprioriWeight(ratio*sizePrior)
    learner.setDatabaseWeight((1-ratio)) #*sizeData)
    learner.useScoreBIC() # or another score with no included prior
    return learner.learnBN()

ratios=[0.0,0.01,0.05,0.2,0.5,0.8,0.9,0.95,0.99,1.0]
gnb.sideBySide(*[learnWithRatio(r) for r in ratios],
              captions=[*[f"with ratio {r}<br/> [datasize : {r*sizePrior+(1-r)*sizeData}]" for r in ratios]])

In [24]:
def learnWithRatio(ratio):
    # bnPrior is used to give the variables and their domains
    learner = gum.BNLearner("database.csv", bnData) 
    learner.useAprioriDirichlet("dirichlet.csv")
    learner.setAprioriWeight(ratio*sizePrior)
    learner.setDatabaseWeight((1-ratio)) #*sizeData)
    learner.useScoreBIC() # or another score with no included prior
    return learner.learnBN()

ratios=[0.0,0.01,0.05,0.2,0.5,0.8,0.9,0.95,0.99,1.0]
gnb.sideBySide(*[learnWithRatio(r) for r in ratios],
              captions=[*[f"with ratio {r}<br/> [datasize : {r*sizePrior+(1-r)*sizeData}]" for r in ratios]])